In [1]:
import os
import sys
os.chdir('..')

In [ ]:
from src.data_preprocessing import load_raw_data, create_match_outcome, merge_team_names, convert_dates
from src.feature_engineering import calculate_league_avg_goals, calculate_recent_goals, merge_team_attributes
from src.utils import inspect_data